In [1]:
import classes
import crossover
import pandas as pd
import random

from deap import base, creator, tools

In [2]:
# Get data from csv file
def get_data(csv_file):
  data_df = pd.read_csv(csv_file, dtype={
    'pos_name': str,
    'latitude': float,
    'longitude': float,
    'address': str,
    'entree_time': int,
    'unloading_time': int,
    'journey2pos_time': int,
    'delivery_time': int,
    'journey2unloadingpoint_time': int,
    'checkout_time': int,
    'min_travels': int,
    'max_travels': int,
    'extra_times': int
  })
  
  data = data_df.to_records(index=False).tolist()
  
  return data

In [3]:
# shuffle the Point_of_Sale list of an individual
def shuffle_sequence(individual):
  random.shuffle(individual)
  return individual

In [4]:
# Read CSV and create a initial list of Point_of_Sale
readed_points_of_sale = get_data('data.csv')

# Create a list of Point_of_Sale
pos_list = [
  classes.Point_of_Sale(
    point[0], point[1], point[2], point[3],
    point[4], point[5], point[6], point[7],
    point[8], point[9], point[10], point[11], point[12]
  )
  for point in readed_points_of_sale
]

In [5]:
# Using DEAP library
# weights is a tuple of -1.0s which means we want to Minimize two values: Distance and Time
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

# initializing DEAP's toolbox
toolbox = base.Toolbox()
# individual's attribute sequence will shuffle a list of POS at creation
toolbox.register("sequence", shuffle_sequence, pos_list)
# individual inherits from Individual and will have the attribute 'sequence'
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.sequence)
# population will be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# mate function from ./crossover.py
toolbox.register("mate", crossover.cxPartialyMatched)
# mutation function that uses DEAP's function mutShuffleIndexes. 
# indpb is the probability of each attribute to be exchanged to another position
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.5)
# tournsize = The number of individuals participating in each tournament
toolbox.register("select", tools.selTournament, tournsize=2)
# fitness function from ./crossover.py
toolbox.register("evaluate", crossover.evaluate)

In [6]:
# Main Genetic Algorithm
def main(matepb, mutpb, ngen, npop):
  """Executes the main genetic algorithm.
  :param matepb: Probability that two individuals cross.
  :param mutpb: Probability that an individual mutates.
  :param ngen: Number of generations the population will have
  :param npop: How many individuals will make up the population
  :returns: a :term:`list` containing the las generation of a population
  """

  population = toolbox.population(n=npop)
  # Evaluate the entire population
  fitnesses = map(toolbox.evaluate, population)

  for individual, fitness in zip(population, fitnesses):
    individual.fitness.values = fitness

  for generation in range(ngen):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < matepb:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    for mutant in offspring:
      if random.random() < mutpb:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    elite = tools.selBest(population, 1)  # Keep the best individual
    # The population is entirely replaced by the offspring
    population[:] = offspring
    population[:1] = elite
    
  return population  

In [7]:
# GA Execution
# (YYYY-MM-DD)T(HH:mm:ss.ms)(UTC-6)
crossover.departure_time = "2024-09-27T09:00:00.000000-06:00"
pop = main(matepb=0.5, mutpb=0.5, ngen=100, npop=100)

In [11]:
# Show results
pd.set_option('display.max_rows', None)
pop_df = pd.DataFrame(pop)

fitnesses_list = []
for ind in pop:
  fitnesses_list.append(ind.fitness.values)

fitness_df = pd.DataFrame(fitnesses_list, columns=['Distance', 'Time'])

result_df = pd.concat([pop_df, fitness_df], axis=1)
result_df.style.hide()

0,1,2,Distance,Time
Avia,Zona4,Majadas,20229.000000,7885.000000
Majadas,Avia,Zona4,23812.000000,8565.000000
Zona4,Majadas,Avia,21972.000000,8449.000000
Zona4,Avia,Majadas,25045.000000,8370.000000
Avia,Zona4,Majadas,20229.000000,7885.000000
Majadas,Zona4,Avia,21787.000000,8002.000000
Avia,Zona4,Majadas,20229.000000,7885.000000
Avia,Zona4,Majadas,20229.000000,7885.000000
Avia,Zona4,Majadas,20229.000000,7885.000000
Avia,Zona4,Majadas,20229.000000,7885.000000


In [10]:
routes_df = pd.DataFrame(crossover.routes_responses_dict).transpose()
routes_df.to_clipboard()
routes_df

,distanceMeters,duration,polyline
"Central,Majadas,2024-07-12T09:20:00.000000-06:00",8344,2433,oe`xA|xlgPq@rCuAy@mAg@gCq@u@WAOZwA?Qh@iBfAiDaD...
"Majadas,Zona4,2024-07-12T10:00:33.000000-06:00",7223,2855,eagxAhyvgPpA{@{AqCvA}@NJR?PIHSASMOQGK@kDwF{@yA...
"Zona4,Avia,2024-07-12T10:48:08.000000-06:00",3878,2010,{tfxAbpmgPwBbBaC`BaE[mEWHa@B{@Ek@i@oCG}@@qCIw@...
"Avia,Central,2024-07-12T11:21:38.000000-06:00",2342,704,o}bxAnlmgPlAwLPmB`P`AjCLjN|@vQ`ARBxH^rAkCn@wAH...
"Central,Avia,2024-07-12T09:20:00.000000-06:00",2653,1806,oe`xA|xlgPq@rCuAy@mAg@gCq@u@WAOZwA?Qh@iBfAiDaD...
"Avia,Majadas,2024-07-12T09:50:06.000000-06:00",7254,2245,o}bxAnlmgPj@uFcHy@I@IJITEF]zGAf@QhCGrAWvC?VTd@...
"Majadas,Zona4,2024-07-12T10:27:31.000000-06:00",7223,2851,eagxAhyvgPpA{@{AqCvA}@NJR?PIHSASMOQGK@kDwF{@yA...
"Zona4,Central,2024-07-12T11:15:02.000000-06:00",5871,1171,{tfxAbpmgPwBbBaC`BaE[qAIZi@LC^?PLDFIdBUp@i@hAa...
"Avia,Zona4,2024-07-12T09:50:06.000000-06:00",2719,2223,o}bxAnlmgPj@uFcHy@I@IJITEF]zGAf@QhCGrAWvC?V_g@...
"Zona4,Majadas,2024-07-12T10:27:09.000000-06:00",6848,2432,{tfxAbpmgPwBbBaC`BaE[qAIZi@LC^?PLDFIdBUp@i@hAa...
